# Neurons

We have now reached one of the main subjects of the course: **neurons**.

A neuron (by which we mean an "artificial neuron") is a caricature of a real biological neuron: it has many inputs $x_1, \ldots, x_n$, and a single output, $y$. FIG

A neuron can thus be represented as a function $f$ with inputs $x_1, \ldots, x_n$ and single output $y$. Usually we take the function to be

$$f_{w,b}(x_1, \ldots, x_n) = \sigma(w_1 x_1 + w_2 x_2 + \cdots + w_n x_n + b).$$

Notice that this is just a more general version of the functions that we have been studying until now. The $w_1, \ldots, w_n$ are called **weights**, and $b$ is known as the **bias**.

Since this is a lot to write, and we don't want to write code with lots of different parameters, we collect all of the $x_1, \ldots, x_n$ into a **vector**, and the $w$s into another vector:

$$
\mathbf{x} = \begin{pmatrix} x_1 \\ \vdots \\ x_n \end{pmatrix};
\qquad
\mathbf{w} = \begin{pmatrix} w_1 \\ \vdots \\ w_n \end{pmatrix}
$$

We thus have

$$f_{\mathbf{w}, b}(\mathbf{x}) = \sigma(\mathbf{w} \cdot \mathbf{x} + b),$$

where the definition of the **dot product** (or scalar product, or inner product) $\mathbf{w} \cdot \mathbf{x}$ is

$$ \mathbf{w} \cdot \mathbf{x} := \sum_i w_i x_i. $$

In Julia, this becomes

In [1]:
σ(x) = 1 / (1 + exp(-x))

f(x, w, b) = σ(w ⋅ x + b)

f (generic function with 1 method)

Note that we can use a syntax in Julia that reflects very closely the mathematics.
    (We could even use bold face if we felt like it.) Here we have implicitly assumed that the user will pass in vectors `x` and `w` to the function `f`, and that they are the same length.
    
[In fact, the function `f` also works with numbers, in which case it is the same as `w*x + b`:

In [2]:
f(3, 4, 5)

0.9999999586006244

In [3]:
σ(3*4 + 5)

0.9999999586006244

## Which weights?

The function `f` maps vectors of length $n$ (in $\mathbb{R}^n$) to numbers (which, due to the definition of $\sigma$, must lie between $0$ and $1$), only once we have chosen particular values for the weights $w_i$ and the bias $b$. How do we choose these?

We will do so as before: we must define a cost function $C(\mathbf{w}, b)$ and minimize this cost function.

The cost function will be provided as follows. We wish to design our function $f$ to model some particular relationship between the input data and the output. 
As a running example, we will use our images of fruit. We could try to design a function $f$ to do the following: take in a picture of the fruit, and output a $0$ if it is an apple, and a $1$ if it is a banana.

In [6]:
using DataFrames

DataFrame(Dict(:A=>[1], :B=>[3]))

,A,B
1,1,3


We will do so later. To start with, we will use summary statistics derived from the images.

In [8]:
# Pkg.add("TextParse")
using TextParse

cols, colnames = TextParse.csvread("apples.dat",'\t')
apples = DataFrame(Dict(name=>col for (name, col) in zip(colnames, cols)))
cols, colnames = TextParse.csvread("bananas.dat",'\t')
bananas = DataFrame(Dict(name=>col for (name, col) in zip(colnames, cols)))


,blue,green,height,red,width
1,0.20750413943355192,0.500662309368192,98,0.5835067538126356,99
2,0.18687105966305895,0.514868541076373,50,0.6096658836445241,99
3,0.18750355493040088,0.5157592947018083,52,0.61001222127424,99
4,0.23931596284375803,0.4929391278094967,99,0.5680333392636676,69
5,0.1833291091603121,0.5148498882514222,51,0.6097662880271572,99
6,0.1833736532775401,0.5146088965412025,53,0.6080886893475433,99
7,0.18620604259534496,0.5154866998805462,53,0.6086763977507786,99
8,0.1885818309426686,0.5164860873506747,53,0.6092775365179625,99
9,0.24194052691261872,0.4945007761667278,99,0.5690986557452848,67
10,0.18911599753288377,0.5161825666623635,54,0.610284435646972,99


These give us `DataFrames` with the data from different images.
We will just use two data points for each image, say the columns 1 and 3, so that each data point $\mathbf{x}^{(i)}$ is a 2-dimensional vector. We also have the label of each point as an apple or a banana, which we call $\mathbf{y}^{(i)}$.

Our neuron will take a point in the two-dimensional plane as argument and try to **classify** it as an apple ($0$) or a banana ($1$). To do so, it must "**learn**" the correct values of the parameters $\mathbf{w}$ and $b$. Note that *in general we cannot expect that this is actually possible*. If it struggles, we may need a more complicated function; see later.

Let's start by putting all the data in a single Julia vector `x` (of which each entry is itself a vector), and the labels in a single vector `y`. [We might need to normalize column 1 to not have huge values. How is this usually dealt with?]

In [9]:
col1 = 3
col2 = 4

x_apples  = [ [apples[i, col1], apples[i, col2]] for i in 1:size(apples)[1] ]
x_bananas = [ [bananas[i, col1], bananas[i, col2]] for i in 1:size(bananas)[1] ]

x = vcat(x_apples, x_bananas)

y = vcat( zeros(size(x_apples)[1]), ones(size(x_bananas)[1]) );

However, the cost function accepts a single two-vector $x$ and the corresponding label $y$.

In [10]:
C(params, x, y) = ( w = params[1:2]; b = params[3]; (y - f(x, w, b))^2 )
#C(w, b) = C([w, b])

C (generic function with 1 method)

We will examine each data point in turn to try to nudge the cost function in the right direction.
We start with *random* parameter values.

In [11]:
using ForwardDiff

In [12]:
w = rand(2)
b = rand()

params = [w; b]

3-element Array{Float64,1}:
 0.366305
 0.221067
 0.599116

IS THIS STOCHASTIC GRADIENT DESCENT?

In [13]:
function gradient_descent(C, x, y, params, N=1000)

    η = 0.01

    for i in 1:N
        
        which = rand(1:length(x))  # choose a data point
        
        xx = x[which]
        yy = y[which]
        
        grad = ForwardDiff.gradient(ws -> C(ws, xx, yy), params)
        #@show grad
        params -= η * grad
    end
    
    return params
    
end
    

gradient_descent (generic function with 2 methods)

In [14]:
w = rand(2)
b = rand()

params = [w; b] ;

@show params
@time params = gradient_descent(C, x, y, params, 1000000)
@show params

params = [0.739475, 0.423584, 0.464474]
 10.648472 seconds (11.47 M allocations: 865.158 MiB, 1.42% gc time)
params = [0.739475, 0.423584, 0.464474]


3-element Array{Float64,1}:
 0.739475
 0.423584
 0.464474

We can check that we have reached a minimum of the cost function, where the gradient should be close to zero:

In [15]:
 ForwardDiff.gradient(ws -> C(ws, x[1], y[1]), params)

3-element Array{Float64,1}:
 1.14236e-27
 6.36503e-30
 1.26928e-29

We can check sample data to see if the function is correctly approximated:

In [16]:
f(x[900], params[1:2], params[3])

1.0

In [17]:
f(x[1], params[1:2], params[3])

1.0

We see that *with sufficient training*, the single neuron is approximately able to learn the function for most of the data, but not all:

In [18]:
maximum(y .- f.(x, [params[1:2]], params[3]))

9.595447658661271e-9

In [19]:
using Plots; gr()

Plots.GRBackend()

In [20]:
scatter(first.(x_apples), last.(x_apples), m=:cross)
scatter!(first.(x_bananas), last.(x_bananas))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0.40 
 
 
 0.45 
 
 
 0.50 
 
 
 0.55 
 
 
 0.60 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<line clip-path="url(#clip0002)" x1="510.544" y1="131.453" x2="510.544" y2="1

Let's draw the function that the network has learned, together with the data:

In [21]:
heatmap(0:0.01:1, 0:0.01:1, (x,y)->f([x,y], params[1:2], params[3]))

scatter!(first.(x_apples), last.(x_apples), m=:cross)
scatter!(first.(x_bananas), last.(x_bananas))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 25 
 
 
 50 
 
 
 75 
 
 
 100 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 0.75 
 
 
 1.00 
 
 
<image width="5" height="343" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAAUAAAFXCAYAAABqXoStAAACcElEQVRYha1YMZLcMAyDXOcH16RO
umSuvc/cv/KwlNfmB5kxkUKSTXqsm4DmlpqRFgBJELvt17d3/nj5wNfvv/Hl5x/8fX3DhpuPcEg2
kA3gBpAATbluAMgGWgNMvr6EZAMWCTQREj0kACZdJ6YgDSBKGBnRGeUgGaYYHZIoyO1hqLsBsF2t
+2REg9w2bDiFBpomyO0hZ92ZFvmoUWHdT0hNpUkgDEJG5PHNYFMFWTLq7biNd2VGpOtmYwGkds47
m2wXOEWGySr1EsNZRpEBogVjzXhyIaTgYHovEbgweiRyhQHasWW2jMgEwGEZXeSKTvammlQprnN1
EAarhCC3h8GT9VbsszlzSZVdAGHHi3Wn82TdALvAU5StjlGApNWdo+45QW4PJ0W/vTKD4DKT7Dbe
k6XrS0gHowQkJzJH5HoMCYybSxc5dnNC5BhMawzQW5AsyPTkckhOaEklInQzpevBrKyybc7iQYfk
E4bMiJc5kq6vGSFumgc1Eq8vIZHt9HqN5mTkI1eixElBPmEUY1y6RlsNJHYD9LlJud4M4f8LVWS7
CLIViGxPWtEwGaW67v7N1mkS/Sd7BpL3ULVt7Ayl44kKRkFkrXCGvY81kDHAxZsRkqgSsY+9iVzb
7MfVxBz9H6McJB/jnkPacY1xssjRxSSzMvg/dCoWH4fIPsZJkKbIqRp9AolZTyb3c96RaJuL25Qw
2hEigsrI4GOc2DaLN3eYg5SY9yPGIcnIxbgika+pRYUEwlAIaYhMZ6yZtkG+bfKCLCBZaBsTW3FA
Ss7mAtLu7ELf7zTMGJey9J4v6ObzKSOMNy3nyXSM9MDzD+qTgEUNCG8PAAAAAElFTkSuQmCC
" transform="translate(53, 22)"/>
 
 
 
 
 
 
 
<image width="18" height="364" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAABIAAAFsCAYAAADBkelSAAACHUlEQVR4nO2c223EMAwEqUD9V5Eu
4yOvBPpjEAwIqoCF9kU5vljnyd8KYP0QIBERN+thgAoC4qhVzqVmcw0Te7BGHLWgkk0BgdQo1yZr
lH8Q0Ca7Wxy1Ux8GKBICOhAQaT+lESQ2aD9Gbe1vgU4mAiSkRgEpk824NjnZRvsh1yj7la6N1Qi0
35ZsnWtC+yORF1pxo3zUMNcYjZTUVqMeCMFBqUFAUGeN1HRA5Ds2CKjyIEBK+xlqt8qnEUSNAiLf
+Y+lFslgTdaImkc3itmUkJpxZm+ym3XLppHwOKI0Eh5HXPt1gw2035Zs0P6x7QeT7aO2rrVAtooY
2++z33aukf8PKSutUKNN9j8C5WrUraXWL18gZ2s09mlk8DzaZPdAQvtDRk1oP/TjupMaZT+Uo8Gl
NdpPAQmpYa7pks2VFhtsOo249iMwS+3NMs5sBEbZ/qXWA/ke2AfPbIya8WEUAtJVZO3vF2g/g6Oc
2XM1AqnZkg0eRxAQtSOlRmt/s4TUVqMXQAyOcmbr5pFu1N6CPqoRJntwaXU5MrZfl+zB1MCfM6Dv
sn3n2s7sF0A6apTYGDWhRkmVVpjswa5R1Ar6etmo0cGo2a4JoXYE2n909nPJttkPltZ2/xGYbOge
LZ/9Oo2WWr/A0kI34GA7Iv/KnksNm0eDNeKo6dq/rvVAVEWoHQmHf+kumcaoBfQmArwngnON0sjX
fs7+udQgoC/rNOfnsknqIwAAAABJRU5ErkJggg==
" transform="translate(532, 12)"/>
 
 
 0.625 
 
 
 0.650 
 
 
 0.675 
 
 
 0.700 
 
 
 0.725 
 
 
 0.750 
 
 
 0.775 
 
 
 0.800 
 
 
 0.825 
 
<polyline clip-path="url(#clip0200)" style="stroke:#000000; stroke-width:1; stroke-opacity:1; fill:none" points="
 550.315,375.869 550.315,357.933 556.315,357.933 550.315,357.933 550.315,316.891 556.315,316.891 550.315,316.891 550.315,275.848 556.315,275.848 550.315,275.848 
 550.315,234.805 556.315,234.805 550.315,234.805 550.315,193.763 556.315,193.763 550.315,193.763 550.315,152.72 556.315,152.72 550.315,152.72 550.315,111.678 
 556.315,111.678 550.315,111.678 550.315,70.6355 556.315,70.6355 550.315,70.6355 550.315,29.5929 556.315,29.5929 550.315,29.5929 550.315,11.811 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

We see that the single neuron has *learnt* to separate the data using something that is close to a hyperplane. (Somehow we restricted the function to be a hyperplane.)